In [153]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import ast
import gzip
from collections import defaultdict
import sys
import operator
from pylmnn.lmnn import LargeMarginNearestNeighbor as LMNN
#from pylmnn.plots import plot_comparison

In [2]:
modClothPath = "C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment2\\modcloth_final_data.json.gz"
print("File path is {}".format(modClothPath))

File path is C:\Users\ramasarma\Documents\UCSD\Fall 2020\CSE 258\Assignment2\modcloth_final_data.json.gz


In [3]:
## Decomposing fit semantics for product size recommendation in metric spaces
def parseDataFromFile(path):
    """ Parse the data and """
    for l in open(path, "rt", encoding="utf-8"):
        yield ast.literal_eval(l)

def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf-8"):
        yield eval(l)

In [4]:
modClothData = list(readGz(modClothPath))

In [5]:
print(len(modClothData))
print(modClothData[0])

82790
{'item_id': '123373', 'waist': '29', 'size': 7, 'quality': 5, 'cup size': 'd', 'hips': '38', 'bra size': '34', 'category': 'new', 'bust': '36', 'height': '5ft 6in', 'user_name': 'Emily', 'length': 'just right', 'fit': 'small', 'user_id': '991571'}


In [6]:
# lengthModClothData = len(modClothData)
# train_val_boundary = int(lengthModClothData * 0.6)
# val_test_boundary = int(lengthModClothData * 0.8)
# training_set = modClothData[:train_val_boundary]
# validation_set = modClothData[train_val_boundary:val_test_boundary]
# test_set = modClothData[val_test_boundary:]

In [7]:
# Let me attempt to find the accuracy of the logisticRegression model
print(len(modClothData))
# We are going to find out the results for the model proposed by Rishabh Misra et. al for this assignment
modClothData[10]

82790


{'item_id': '123373',
 'size': 24,
 'quality': 4,
 'cup size': 'd',
 'hips': '49.0',
 'bra size': '40',
 'category': 'new',
 'length': 'just right',
 'height': '5ft 6in',
 'user_name': 'micorson',
 'fit': 'small',
 'user_id': '205796'}

In [67]:
random.seed(0)
random.shuffle(modClothData)

trainData = modClothData[:int(0.8*len(modClothData))]
valData = modClothData[int(0.8*len(modClothData)):int(0.9*len(modClothData))]
testData = modClothData[int(0.9*len(modClothData)):]

print(len(trainData), len(valData), len(testData))

66232 8279 8279


### User Item pairs data is going to be collected here

In [68]:
itemReviews = defaultdict(list)
userReviews = defaultdict(list)
itemIndex = {}
userIndex = {}
item_i, user_i = 0, 0
# The key used to access the dictionary would be 
for d in trainData:
    item_key = d['item_id'] + '|' + str(d['size'])
    user_key = d['user_id']
    #
    if item_key not in itemReviews:
        itemReviews[item_key] = [d]
        itemIndex[item_key] = item_i
        item_i += 1
    else:
        itemReviews[item_key].append(d)
    
    if user_key not in userReviews:
        userReviews[user_key] = [d]
        userIndex[user_key] = user_i
        user_i += 1
    else:
        userReviews[user_key].append(d)
    
print(len(userReviews), len(itemReviews), len(userIndex), len(itemIndex))

41212 4578 41212 4578


### Basic Logistic Regression model with multi-class classifier

In [200]:
def performOneHotEncoding(attribute, value):
    """ Hips, category and fit"""
    hip_bins = [-2, 0, 30, 35, 40, 45]
    categories = ["new", "dresses", "tops", "bottoms", "outerwear", "sale", "wedding"]
    hip_vector = [0] * len(hip_bins)
    category_vector = [0] * (len(categories))
    if(attribute == "hips"):
        if(value <= -2):
            hip_vector[0] = 0
        elif (value >= 0 and value < 30):
            hip_vector[0] = 1
        elif (value >= 30 and value < 35):
            hip_vector[1] = 1
        elif (value >= 35 and value < 40):
            hip_vector[2] = 1
        elif (value >= 40 and value < 45):
            hip_vector[3] = 1
        elif (value >= 45):
            hip_vector[4] = 1
        return hip_vector
    elif (attribute == "category"):
        if(value == "new"):
            category_vector[0] = 1
        elif (value == "dresses"):
            category_vector[1] = 1
        elif (value == "tops"):
            category_vector[2] = 1
        elif (value == "bottoms"):
            category_vector[3] = 1
        elif (value == "outerwear"):
            category_vector[4] = 1
        elif (value == "sale"):
            category_vector[5] = 1
        elif (value == "wedding"):
            category_vector[6] = 1
        return category_vector
    return None

def convertHeight(height):
    """ Converts height to a metric scale"""
    if(len(height) > 0 and len(height) <= 4):
        return int(height[0]) * 30.48
    else:
        return (int(height[0]) * 30.48) + (int(height[4:-2])*2.54)
        
def extractFeatures(dataset):
    features = []
    # Set of features that we have decided for this problem are
    # 1. Bra size
    # 2. Hips (Converted to a category)
    # 3. Shoe size ()
    # 4. Height
    # 5. First time user
    # 6. Size
    # 7. Category (new, dresses, bottoms)
        
    X_train = []; Y_train = []
    for d in dataset:
        fe = []
        if('hips' in d):
            hip_vector = performOneHotEncoding("hips", float(d['hips']))
        else:
            hip_vector = performOneHotEncoding("hips", -10)
        
        category_vector = performOneHotEncoding("category", d['category'])
        # Is height actually present in the given datasample?
        if('height' in d):
            new_height = convertHeight(d['height'])
        else:
            new_height = 0
        # The following values have been captured here
        if('bra size' in d):
            fe.append(float(d['bra size']))
        else:
            fe.append(0)
        fe = fe + hip_vector
        fe = fe + category_vector
        fe.append(float(d['size']))
        fe.append(new_height)
        X_train.append(fe)
        if 'small' in d['fit']:
            Y_train.append(0)
        elif 'fit' in d['fit']:
            Y_train.append(1)
        elif 'large' in d['fit']:
            Y_train.append(2)
    
    return (X_train, Y_train)


def compute_confusion_matrix(Y_actual, Y_pred):
    """ Computes the confusion matrix for this project """
    #print(type(Y_actual), type(Y_pred))
    confusion_matrix = [[0]*3]*3
    #print(confusion_matrix)
    for i in range(len(Y_actual)):
        #print(Y_pred[i], Y_actual[i], type(Y_pred[i]), type(Y_actual[i]))
        confusion_matrix[Y_pred[i]][Y_actual[i]] += 1
    return confusion_matrix

X_train, Y_train = extractFeatures(trainData)
X_test, Y_test = extractFeatures(testData)
X_val, Y_val = extractFeatures(valData)
print(X_train[0], Y_train[0])
print(X_test[0], Y_test[0])
lambdas = [10 **i for i in np.arange(-3,4,dtype=float)]
confusion_matrices = {}
test_labels_auc = []
 
for val in Y_test:
    label = [0] * 3
    if(val == 0):
        label[0] = 1
    elif(val == 1):
        label[1] = 1
    elif(val == 2):
        label[2] = 1
    test_labels_auc.append(label)

test_labels_auc = np.array(test_labels_auc)

def kFoldCrossValidation(dataset, K):
    random.shuffle(dataset)
    

for lamda in lambdas:
    clf_1 = LogisticRegression(fit_intercept=True,multi_class='ovr',C=lamda)
    clf_1.fit(X_train,Y_train)
    pred = clf_1.predict_proba(X_test)
    predictions = []
    for i in range(len(pred)):
        label = 0
        for j in range(len(pred[i])):
            if(pred[i][j] > pred[i][label]):
                label = j
        predictions.append(label)
    auc = []
    for i in range(3):
        auc.append(roc_auc_score(test_labels_auc[:, i], pred[:, i], average='weighted'))
    print("LAMBDA = {} | Average AUC is {} and auc = {}".format(lamda, np.mean(auc), auc))
    confusion_matrix = compute_confusion_matrix(Y_test, predictions)
    confusion_matrices[lamda] = confusion_matrix

[34.0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 15.0, 172.72] 1
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4.0, 175.26] 1
LAMBDA = 0.001 | Average AUC is 0.5798515011002063 and auc = [0.5833485025276898, 0.5708062757434603, 0.5853997250294689]
LAMBDA = 0.01 | Average AUC is 0.5806869158618884 and auc = [0.5860903071248942, 0.5704066455866814, 0.5855637948740895]
LAMBDA = 0.1 | Average AUC is 0.5796519369476255 and auc = [0.5852210795490013, 0.5700093741239136, 0.5837253571699619]
LAMBDA = 1.0 | Average AUC is 0.5799620953044361 and auc = [0.585302382065516, 0.5700690153867718, 0.5845148884610205]
LAMBDA = 10.0 | Average AUC is 0.5801104717170101 and auc = [0.5857394512985922, 0.5699409719975846, 0.5846509918548535]
LAMBDA = 100.0 | Average AUC is 0.5805860685884773 and auc = [0.5851369396292255, 0.570352058668133, 0.5862692074680733]
LAMBDA = 1000.0 | Average AUC is 0.580564146011867 and auc = [0.5862859788190029, 0.5706150867460267, 0.5847913724705716]


In [ ]:
#Here, we compute the values for X_val and Y_val
clf_1 = LogisticRegression(fit_intercept=True,multi_class='ovr',C=0.01)
clf_1.fit(X_train, Y_train)
clf_1.predict(X_val, Y_val)

### Here, we will implement Training for the given model

In [69]:
# Here, we will compute the latent customer and product sizes for this model
latentCustomerSize = np.zeros(len(userReviews))
latentProductSize = np.zeros(len(itemReviews))

w = 1; b1 = -1; b2 = 1; lamda = 0.1

# We have the latent product size
for item in itemReviews:
    latentProductSize[itemIndex[item]] = int(item.split('|')[1])

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

def encodeLabel(dataPoint):
    try:
        fit = dataPoint['fit']
        if(fit == 'small'):
            return 0
        elif (fit == 'fit'):
            return 1
        elif (fit == 'large'):
            return 2
    # Handling an exception in this case through the keyword "e"
    except Keyerror as e:
        print(e.message)
        
def val(s, t):
    """
        This function computes the difference between customer size and product size
    """
    return w * (s - t)

# The main purpose for modeling the loss in this way isp primarily due to intuition
# that for any customer size, product size = s, t
# s - t > b1 ==> (Product is small)
# b2 < s - t < b1 ==> (Product is fit)
# s - t < b2 ==> (Product is large)

def userLoss(user, customerSize):
    """
        This function computes the hinge loss for a user u
    """
    loss = 0
    for d in userReviews[user]:
        if 'small' in d['fit']:
            loss += max(0, 1 - val(customerSize, latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]]) + b2)
        elif 'fit' in d['fit']:
            loss += max(0, 1 + val(customerSize, latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]]) - b2)
            loss += max(0, 1 - val(customerSize, latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]]) + b1)
        elif 'large' in d['fit']:
            loss += max(0, 1 + val(customerSize, latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]]) - b1)
    return loss

def itemLoss(item, productSize):
    """
        This function computes the hinge loss for an item i
    """
    loss = 0
    for d in itemReviews[item]:
        if 'small' in d['fit']:
            loss += max(0, 1 - val(latentCustomerSize[userIndex[d['user_id']]], productSize) + b2)
        elif 'fit' in d['fit']:
            loss += max(0, 1 + val(latentCustomerSize[userIndex[d['user_id']]], productSize) - b2)
            loss += max(0, 1 - val(latentCustomerSize[userIndex[d['user_id']]], productSize) + b1)
        elif 'large' in d['fit']:
            loss += max(0, 1 + val(latentCustomerSize[userIndex[d['user_id']]], productSize) - b1)
    return loss

def totalLoss():
    """
        Here, we compute the total loss for all items
    """
    loss = 0
    for item in itemReviews:
        for d in itemReviews[item]:
            productSize = latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]]
            if 'small' in d['fit']:
                loss += max(0, 1 - val(latentCustomerSize[userIndex[d['user_id']]], productSize) + b2)
            elif 'fit' in d['fit']:
                loss += max(0, 1 + val(latentCustomerSize[userIndex[d['user_id']]], productSize) - b2)
                loss += max(0, 1 - val(latentCustomerSize[userIndex[d['user_id']]], productSize) + b1)
            elif 'large' in d['fit']:
                loss += max(0, 1 + val(latentCustomerSize[userIndex[d['user_id']]], productSize) - b1)
    return loss

def AUC():
    X_train = []
    Y_train = []
    # For every datapoint in the training data
    for d in trainData:
        fe = []
        fe.append(latentCustomerSize[userIndex[d['user_id']]])
        item_key = d['item_id'] + '|' + str(d['size'])
        fe.append(latentProductSize[itemIndex[item_key]])
        X_train.append(fe)
        
        # Encoding the fit values
        if 'small' in d['fit']:
            Y_train.append(0)
        elif 'fit' in d['fit']:
            Y_train.append(1)
        elif 'large' in d['fit']:
            Y_train.append(2)
    
    # This is the model used for fitting the extracted features
    # Multiple class based, One class over rest type of Logistic Regression model
    # has been used here
    c = 1
    clf_1 = LogisticRegression(fit_intercept=True,multi_class='ovr',C=c)
    clf_1.fit(X_train,Y_train)
    
    # The following are the examples of the test-set for both X and Y
    X_test = []
    Y_test = []
    test_labels_auc = []
    for d in testData:
        fe = []
        try:
            fe.append(latentCustomerSize[userIndex[d['user_id']]])
        except KeyError:
            fe.append(np.mean(latentCustomerSize))
        
        try:
            fe.append(latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]])
        except KeyError:
            fe.append(np.mean(latentProductSize))
    
        X_test.append(fe)
        
        # Encoding the actual test set values
        # Encoding the value into the label
        label = [0, 0, 0]
        if 'small' in d['fit']:
            Y_test.append(0)
            label[0] = 1
        elif 'fit' in d['fit']:
            Y_test.append(1)
            label[1] = 1
        elif 'large' in d['fit']:
            Y_test.append(2)
            label[2] = 1
        
        test_labels_auc.append(label)
    
    
    test_labels_auc = np.array(test_labels_auc)
    pred = clf_1.predict_proba(X_test)
    auc = []
    for i in range(3):
        auc.append(roc_auc_score(test_labels_auc[:, i], pred[:, i], average='weighted'))
    print("Average AUC is {} and auc = {}".format(np.mean(auc), auc))

In [71]:
prevLoss = sys.maxsize
currLoss = 0
for iterr in range(0, 200):
    # Re-computing the values needed for every step in Regression
    for user in userReviews:
        candidateSizes = []
        for d in userReviews[user]:
            if 'small' in d['fit']:
                candidateSizes.append(latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]] + ((b2 + 1)/w))
            elif 'fit' in d['fit']:
                candidateSizes.append(latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]] + ((b2 - 1)/w))
                candidateSizes.append(latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]] + ((b1 + 1)/w))
            elif 'large' in d['fit']:
                candidateSizes.append(latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]] + ((b1 - 1)/w))
        flag = 0
        candidateSizes = list(set(candidateSizes))
        candidateSizes = sorted(candidateSizes)
        
        if(len(candidateSizes) == 1):
            latentCustomerSize[userIndex[user]] = candidateSizes[0]
        else:
            for s in range(1, len(candidateSizes)):
                slope = (userLoss(user, candidateSizes[s]) - userLoss(user, candidateSizes[s-1])) /(candidateSizes[s] - candidateSizes[s-1])
                if slope >= 0:
                    flag = 1
                    latentCustomerSize[userIndex[user]] = candidateSizes[s - 1]
                    break
            if flag == 0:
                latentCustomerSize[userIndex[user]] = candidateSizes[-1]
    # Phase - 2
    for item in itemReviews:
        candidateSizes = []
        for d in itemReviews[item]:
            if 'small' in d['fit']:
                candidateSizes.append(latentCustomerSize[userIndex[d['user_id']]] + ((b2 + 1)/w))
            elif 'fit' in d['fit']:
                candidateSizes.append(latentCustomerSize[userIndex[d['user_id']]] + ((b2 - 1)/w))
                candidateSizes.append(latentCustomerSize[userIndex[d['user_id']]] + ((b1 + 1)/w))
            elif 'large' in d['fit']:
                candidateSizes.append(latentCustomerSize[userIndex[d['user_id']]] + ((b1 - 1)/w))
                
        flag = 0
        candidateSizes = list(set(candidateSizes))
        candidateSizes = sorted(candidateSizes)
        
        if(len(candidateSizes) == 1):
            latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]] = candidateSizes[0]
        else:
            for s in range(1, len(candidateSizes)):
                slope = (itemLoss(item, candidateSizes[s]) - itemLoss(item, candidateSizes[s-1])) / (candidateSizes[s] - candidateSizes[s-1])
                if(slope >= 0):
                    flag = 1
                    latentProductSize[itemIndex[item]] = candidateSizes[s - 1]
                    break
            if(flag == 0):
                latentProductSize[itemIndex[item]] = candidateSizes[-1]
                
    # Phase - 3
    learning_rate = 0.0000005/np.sqrt(iterr + 1)
    grad_w, grad_b1, grad_b2 = 0, 0, 0
    for d in trainData:
        s = latentCustomerSize[userIndex[d['user_id']]]
        t = latentProductSize[itemIndex[d['item_id'] + '|' + str(d['size'])]]
        
        if 'small' in d['fit']:
            A = 1 - val(s, t) + b2
            if A > 0:
                grad_w += -1 * (s - t)
                grad_b2 += 1
        elif 'fit' in d['fit']:
            B = 1 + val(s, t) - b2
            C = 1 - val(s, t) + b1
            if B > 0:
                grad_w += 1 * (s - t)
                grad_b2 -= -1
            if C > 0:
                grad_w += -1 * (s - t)
                grad_b1 += 1
        elif 'large' in d['fit']:
            D = 1 + val(s, t) - b1
            if D > 0:
                grad_w += (s - t)
                grad_b1 += -1
        
    w = w - learning_rate * (grad_w + 2*lamda*w)
    b1 = b1 - learning_rate * (grad_b1 + 2*lamda*b1)
    b2 = b2 - learning_rate * (grad_b2 + 2*lamda*b2)
    
    if(iterr == 0):
        currLoss = totalLoss()
    else:
        prevLoss = currLoss
        currLoss = totalLoss()
    
    if(iterr % 2 == 0):
        print("Iteration = {} and total loss = {}".format(iterr, currLoss))
        AUC()
    
    if(currLoss - prevLoss > 10000):
        break    

Iteration = 0 and total loss = 49454.13267050461
Average AUC is 0.537575487369578 and auc = [0.49855074171903857, 0.5439152770589377, 0.5702604433307582]
Iteration = 2 and total loss = 81647.14973745184
Average AUC is 0.5404118402430634 and auc = [0.500606604077916, 0.5492969744021048, 0.5713319422491694]
Iteration = 4 and total loss = 76389.90933100629
Average AUC is 0.5405006976922129 and auc = [0.5017290153293985, 0.548832446464385, 0.5709406312828551]
Iteration = 6 and total loss = 74492.41272235464
Average AUC is 0.5426810787507123 and auc = [0.5045721479622861, 0.5459260973992366, 0.5775449908906141]
Iteration = 8 and total loss = 86283.82975326688
Average AUC is 0.5421219174966562 and auc = [0.5001820848977248, 0.55174580422031, 0.5744378633719336]
Iteration = 10 and total loss = 91041.76443555835
Average AUC is 0.5516388769885863 and auc = [0.512868278573633, 0.5671048446227168, 0.5749435077694091]
Iteration = 12 and total loss = 91020.86001665253
Average AUC is 0.5537038962242

Average AUC is 0.5883725374804166 and auc = [0.5853676423405172, 0.5783735052619094, 0.6013764648388233]
Iteration = 108 and total loss = 45836.24386146317
Average AUC is 0.5863877071735458 and auc = [0.5815241613618138, 0.5791730351404973, 0.5984659250183262]
Iteration = 110 and total loss = 45920.83991092307
Average AUC is 0.5831960575962473 and auc = [0.5769929374850218, 0.5768790704319511, 0.5957161648717688]
Iteration = 112 and total loss = 46106.9215212512
Average AUC is 0.5811502688440952 and auc = [0.5746646844130963, 0.5763975262017209, 0.5923885959174684]
Iteration = 114 and total loss = 45750.89022221721
Average AUC is 0.5831462963978712 and auc = [0.5752431872856396, 0.5805123689913955, 0.5936833329165784]
Iteration = 116 and total loss = 45546.65952520279
Average AUC is 0.5825078146624288 and auc = [0.5726227908087341, 0.5795041283884323, 0.5953965247901201]
Iteration = 118 and total loss = 45938.84074927921
Average AUC is 0.5838540985288819 and auc = [0.5808799616906908, 

### Performance on the validation set

In [76]:
# Let us have this model tested on the test set
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

X_train = []; Y_train = []
for r in trainData:
    fe = []
    fe.append(latentCustomerSize[userIndex[r['user_id']]])
    fe.append(latentProductSize[itemIndex[r['item_id'] + '|' + str(r['size'])]])
    X_train.append(fe)

    if 'small' in r['fit']:
        Y_train.append(0)
    elif 'fit' in r['fit']:
        Y_train.append(1)
    elif 'large' in r['fit']:
        Y_train.append(2)

c = 1
# Here, we have a classifier with the following settings
clf_2 = LogisticRegression(fit_intercept=True, multi_class='ovr', C=c)
clf_2.fit(X_train, Y_train)

X_val = []; Y_val = []; val_labels_auc = []
for r in valData:
    fe = []
    
    try:
        u = userIndex[r['user_id']]
        fe.append(latentCustomerSize[u])
    except KeyError:
        fe.append(np.mean(latentCustomerSize))
        
    try:
        fe.append(latentProductSize[itemIndex[r['item_id'] + '|' + str(r['size'])]])
    except KeyError:
        fe.append(np.mean(latentProductSize))

    X_val.append(fe)
    label = [0, 0, 0]
    if 'small' in r['fit']:
        Y_val.append(0)
        label[0] = 1
    elif 'fit' in r['fit']:
        Y_val.append(1)
        label[1] = 1
    elif 'large' in r['fit']:
        Y_val.append(2)
        label[2] = 1
    
    val_labels_auc.append(label)

val_labels_auc = np.array(val_labels_auc)

pred = clf_2.predict_proba(X_val)
AUC = []
for i in range(3):
    AUC.append(roc_auc_score(val_labels_auc[:,i], pred[:,i], average='weighted'))
print('Average AUC', np.mean(AUC), AUC)

Average AUC 0.5483376954870677 [0.5794967150204958, 0.5351525306651438, 0.5303638407755635]


### Implementing a model with "K" latent factors and using Logistic Regression on top of it to achieve the best performance

In [154]:
data = modClothData

random.seed(1)
random.shuffle(data)

train_data = modClothData[:int(0.8*len(data))]
val_data = modClothData[int(0.8*len(data)):int(0.9*len(data))]
test_data = modClothData[int(0.9*len(data)):]

In [155]:
item_data = {}; user_data = {}
item_index = {}; user_index = {}; p_item_index = {} ## p_ is for parent 
u_index = 0; i_index = 0; p_i_index = 0
sizes = {}
s_index = 0

for r in data:
    if r['size'] not in sizes:
        sizes[r['size']] = s_index
        s_index += 1

    if r['item_id'] + '|' + str(r['size']) not in item_data:
        item_data[r['item_id'] + '|' + str(r['size'])] = [r]
        item_index[r['item_id'] + '|' + str(r['size'])] = i_index
        i_index += 1
    else:
        item_data[r['item_id'] + '|' + str(r['size'])].append(r)
        
    if r['item_id'] not in p_item_index:
        p_item_index[r['item_id']] = p_i_index
        p_i_index += 1
        
    if r['user_id'] not in user_data:
        user_data[r['user_id']] = [r]
        user_index[r['user_id']] = u_index
        u_index += 1
    else:
        user_data[r['user_id']].append(r)
len(user_data), len(user_index), len(item_data), len(item_index), len(p_item_index)

(47958, 47958, 5005, 5005, 1378)

The above statistic implies that there are only 1302 parent products in this dataset and amongst the 4578 unique item + size keys, 3576 are just variants of the other 1302 parent products.

In [170]:
product_sizes = {}; product_sizes_rev = {}

for r in data:
    if r['item_id'] not in product_sizes:
        product_sizes[r['item_id']] = [r['size']]
    else:
        product_sizes[r['item_id']].append(r['size'])
        
for k in product_sizes:
    product_sizes[k] = list(sorted(set(product_sizes[k])))
    product_sizes_rev[k] = list(sorted(set(product_sizes[k]), reverse=True))

# Here, product_smaller and product_larger refer to the two
# dictionaries used to store the smaller and larger products
product_smaller = {}
product_larger = {}

for k in product_sizes:
    for i in range(len(product_sizes[k])):
        if len(product_sizes[k]) == 1:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = -1
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = -1
        elif i == 0:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = -1
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i+1])]
        elif i == len(product_sizes[k]) - 1:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i-1])]
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = -1
        else:
            product_smaller[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i-1])]
            product_larger[item_index[k + '|' + str(product_sizes[k][i])]] = item_index[k + '|' + str(product_sizes[k][i+1])]

In [171]:
def f(a,b_i,b_u,s,t):
    return np.dot(w, np.concatenate(([a, b_u, b_i], np.multiply(s, t))))

# Random initialization for the K fold model
K = 10; learning_rate = 0.000005; alpha = 1; 
true_size_item = np.random.normal(size = (len(item_index), K))*0.1
true_size_cust = np.random.normal(size = (len(user_index), K), loc=1, scale=0.1)
bias_i = np.random.normal(size = (len(item_index)))*0.1
bias_u = np.random.normal(size = (len(user_index)))*0.1
b_1 = -5; b_2 = 5; lamda = 2; w = np.ones(K+3)

for k in product_sizes:
    start = len(product_sizes[k])
    for size in product_sizes[k]:
        true_size_item[item_index[k + '|' + str(size)]] = np.random.normal(size=(1,K), loc=start, scale=0.1)
        start -= 1

In [172]:
def calc_auc():
    train_features = []; train_labels = []
    for r in train_data:
        fe = []
        fe.append(w[1]*bias_u[user_index[r['user_id']]])
        fe.append(w[2]*bias_i[p_item_index[r['item_id']]])
        fe.extend(np.multiply(w[3:], np.multiply(true_size_cust[user_index[r['user_id']]], true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])))

        train_features.append(fe)

        if 'small' in r['fit']:
            train_labels.append(1)
        elif 'fit' in r['fit']:
            train_labels.append(2)
        elif 'large' in r['fit']:
            train_labels.append(3)

    clf = LogisticRegression(fit_intercept=True, multi_class='ovr')
    clf.fit(train_features, train_labels)

    test_features = []; test_labels = []; test_labels_auc = []
    for r in test_data:
        fe = []
        flag = 0
        fe.append(w[1]*bias_u[user_index[r['user_id']]])
        fe.append(w[2]*bias_i[p_item_index[r['item_id']]])
        fe.extend(np.multiply(w[3:], np.multiply(true_size_cust[user_index[r['user_id']]], true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])))

        test_features.append(fe)
        label = [0, 0, 0]
        if 'small' in r['fit']:
            test_labels.append(1)
            label[0] = 1
        elif 'fit' in r['fit']:
            test_labels.append(2)
            label[1] = 1
        elif 'large' in r['fit']:
            test_labels.append(3)
            label[2] = 1
        test_labels_auc.append(label)

    test_labels_auc = np.array(test_labels_auc)

    from sklearn.metrics import roc_auc_score
    pred = clf.predict_proba(test_features)
    AUC = []
    for i in range(3):
        AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))
    print('Average AUC', np.mean(AUC), AUC)

In [173]:
def calc_metric_auc():
    U = true_size_cust; V = true_size_item
    def prepare_features(data):
        X = []
        Y = []
        Y_auc = []
        item_l = []
        item_n = []
        items = {}
        item_count = defaultdict(int)
        item_small = defaultdict(int)
        frac_small = []
        for r in data:
            fe = []
            fe.append(w[1]*bias_u[user_index[r['user_id']]])
            fe.append(w[2]*bias_i[p_item_index[r['item_id']]])
            fe.extend(np.multiply(w[3:], np.multiply(true_size_cust[user_index[r['user_id']]], true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])))
            X.append(fe)
            item_l.append(np.multiply(w[3:],V[item_index[r['item_id'] + '|' + str(r['size'])]]))
            item_n.append(str(r['category']))
            items[r['item_id'] + '|' + str(r['size'])] = 1
            if 'small' in r['fit']:
                Y.append(0)
                Y_auc.append([1,0,0])
            elif 'fit' in r['fit']:
                Y.append(1)
                Y_auc.append([0,1,0])
            else:
                Y.append(2)
                Y_auc.append([0,0,1])

        for r in train_data:
            if (r['item_id'] + '|' + str(r['size'])) in items:
                item_count[r['item_id'] + '|' + str(r['size'])] += 1
                if 'small' in r['fit']:
                    item_small[r['item_id'] + '|' + str(r['size'])] += 1
        for k in item_count:
            item_small[k] = item_small[k]/item_count[k]

        for r in data:
            frac_small.append(item_small[r['item_id'] + '|' + str(r['size'])])

        return np.array(X),np.array(Y), np.array(Y_auc), np.array(item_l), item_n, frac_small
    
    def select_prototype(data):
        chosen_data = []
        X_small,_,_,_,_,_ = prepare_features(data)
        small_mean = np.mean(X_small, axis = 0)
        dist = []
        for i in range(len(data)):
            dist.append((i, sum([(X_small[i][j]-small_mean[j])**2 for j in range(len(small_mean))])))
            
        if 'fit' in data[0]['fit']:
            small_temp = sorted(dist, key=operator.itemgetter(1))[1300:]
            interval = int(len(data)/5000)
            for i in range(10):
                #print("Length of small_temp = {} and index = {}".format(len(small_temp), int(i * interval)))
                chosen_data.append(data[small_temp[int(i*interval)][0]])
                interval += 4
        elif 'small' in data[0]['fit']:
            small_temp = sorted(dist, key=operator.itemgetter(1))[1200:]
            interval = int(len(data)/2000)
            for i in range(10):
                #print("Length of small_temp = {} and index = {}".format(len(small_temp), int(i * interval)))
                chosen_data.append(data[small_temp[int(i*interval)][0]])
                interval += 1.15
        else:
            small_temp = sorted(dist, key=operator.itemgetter(1))[1200:]
            interval = int(len(data)/500)
            for i in range(10):
                #print("Length of small_temp = {} and index = {}".format(len(small_temp), int(i * interval)))
                chosen_data.append(data[small_temp[int(i*interval)][0]])
                interval += 0.5

        return chosen_data

    small = []; true = []; large= []
    for r in range(len(train_data)):
        if 'small' in train_data[r]['fit']:
            small.append(train_data[r])
        elif 'fit' in train_data[r]['fit']:
            true.append(train_data[r])
        else:
            large.append(train_data[r])

    random.shuffle(small); random.shuffle(true); random.shuffle(large)
    data_training = [];
    data_training.extend(select_prototype(small))
    data_training.extend(select_prototype(true))
    data_training.extend(select_prototype(large))

    random.shuffle(data_training)
    X_train, Y_train, Y_train_auc, item_l, item_n, frac_small = prepare_features(data_training)
    X_test, Y_test, Y_test_auc, _, _, _ = prepare_features(test_data)

    clf_kLF = LMNN(n_neighbors=29, max_iter=50, n_components=X_train.shape[1], verbose=0)
    clf_kLF = clf_kLF.fit(X_train, Y_train)

    from sklearn.metrics import roc_auc_score
    pred = clf_kLF.transform(X_test); AUC = []
    for i in range(3):
        AUC.append(roc_auc_score(Y_test_auc[:,i], pred[:,i], average='weighted'))
    print('Average AUC', np.mean(AUC), AUC)
    return np.mean(AUC)

In [174]:
def total_loss():
    loss = 0
    for item in item_data:
        for r in item_data[item]:
            s = true_size_cust[user_index[r['user_id']]]
            t = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]
            b_i = bias_i[p_item_index[r['item_id']]]
            b_u = bias_u[user_index[r['user_id']]]
            
            if 'small' in r['fit']:
                loss += max(0, 1 - f(alpha,b_i,b_u,s,t) + b_2)
            elif 'fit' in r['fit']:
                loss += max(0, 1 + f(alpha,b_i,b_u,s,t) - b_2)
                loss += max(0, 1 - f(alpha,b_i,b_u,s,t) + b_1)
            elif 'large' in r['fit']:
                loss += max(0, 1 + f(alpha,b_i,b_u,s,t) - b_1)
    return loss

for iterr in range(0,1000):
    learning_rate1 = 0.00025
    learning_rate2 = 0.00001/np.sqrt(iterr+1)
    
    grad_b1 = 0
    grad_b2 = 0
    grad_s = np.zeros((len(user_index), K))
    grad_t = np.zeros((len(item_index), K))
    grad_bu = np.zeros(len(user_index))
    grad_bi = np.zeros(len(p_item_index))
    grad_bs = np.zeros(len(sizes))
    grad_w = np.zeros((K+3))
    for r in train_data:
        s = true_size_cust[user_index[r['user_id']]]
        t = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]
        b_i = bias_i[p_item_index[r['item_id']]]
        b_u = bias_u[user_index[r['user_id']]]
        
        if 'small' in r['fit']:
            A = 1 - f(alpha,b_i,b_u,s,t) + b_2
            if A>0:
                grad_s[user_index[r['user_id']]] += -1*np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += -1*np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += -1*w[1]
                grad_bi[p_item_index[r['item_id']]] += -1*w[2]
                grad_w += -1*np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b2 += 1
        elif 'fit' in r['fit']:
            B = 1 + f(alpha,b_i,b_u,s,t) - b_2
            C = 1 - f(alpha,b_i,b_u,s,t) + b_1
            if B>0:
                grad_s[user_index[r['user_id']]] += np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += w[1]
                grad_bi[p_item_index[r['item_id']]] += w[2]
                grad_w += np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b2 += -1
            if C>0:
                grad_s[user_index[r['user_id']]] += -1*np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += -1*np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += -1*w[1]
                grad_bi[p_item_index[r['item_id']]] += -1*w[2]
                grad_w += -1*np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b1 += 1
        elif 'large' in r['fit']:
            D = 1 + f(alpha,b_i,b_u,s,t) - b_1
            if D>0:
                grad_s[user_index[r['user_id']]] += np.multiply(w[3:],t)
                grad_t[item_index[r['item_id'] + '|' + str(r['size'])]] += np.multiply(w[3:],s)
                
                grad_bu[user_index[r['user_id']]] += w[1]
                grad_bi[p_item_index[r['item_id']]] += w[2]
                grad_w += np.concatenate(([alpha, b_u, b_i], np.multiply(s, t)))
                grad_b1 += -1

    for i in range(len(user_index)):
        true_size_cust[i] -= learning_rate1*(grad_s[i] + 2*lamda*true_size_cust[i])
        bias_u[i] -= learning_rate1*(grad_bu[i] + 2*lamda*bias_u[i])
        
    for i in range(len(item_index)):
        ## constraint update
        temp = true_size_item[i] - learning_rate1*(grad_t[i] + 2*lamda*true_size_item[i])
        if product_smaller[i] != -1:
            temp = np.maximum(temp, true_size_item[product_smaller[i]])
        if product_larger[i] != -1:
            temp = np.minimum(temp, true_size_item[product_larger[i]])
        true_size_item[i] = temp
        
    for i in range(len(p_item_index)):
        bias_i[i] -= learning_rate1*(grad_bi[i] + 2*lamda*bias_i[i])        
    
    b_1 -= learning_rate2*(grad_b1 + 2*lamda*b_1)
    b_2 -= learning_rate2*(grad_b2 + 2*lamda*b_2)
    w -= learning_rate2*(grad_w + 2*lamda*w)
    if iterr%5 == 0:    
        print(iterr, total_loss(), b_1, b_2,w[:4])
        calc_auc()
        auc = calc_metric_auc()

0 5315875.961330514 -4.89548 5.45462 [ 0.44082     0.99915062  0.9959445  -1.97741958]
Average AUC 0.5581561959585731 [0.5540493673245513, 0.5655556276216276, 0.5548635929295402]
Average AUC 0.5184489401607896 [0.4444487733032656, 0.5627537525641019, 0.5481442946150015]
5 164755.2168482925 -5.262500067410277 5.43848479908934 [ 0.82386968  0.99919769  0.94013484 -0.10273569]
Average AUC 0.5593996445036905 [0.5774080714082731, 0.5108260808251202, 0.5899647812776783]
Average AUC 0.5186869892507078 [0.543671019243269, 0.49647309839315384, 0.5159168501157005]
10 159096.91801233278 -5.0866227885569 5.264513777541331 [ 0.82189613  0.99948856  0.95706029 -0.0988002 ]
Average AUC 0.5886333682648281 [0.6222642521674486, 0.5169232069318351, 0.6267126456952008]
Average AUC 0.5071452936445229 [0.5733158811206608, 0.5038245554166265, 0.4442954443962813]
15 154265.51313694112 -4.946409097736585 5.125819801912092 [ 0.82032277  1.00010842  0.98242991 -0.09594463]
Average AUC 0.6007180993541779 [0.64005

Average AUC 0.5933881509600544 [0.6346106668028622, 0.5123957606131346, 0.6331580254641664]
Average AUC 0.4998640223899195 [0.34667764086943276, 0.49386553260385724, 0.6590488936964686]
145 94673.99365872347 -3.477957653981243 3.673544734447838 [ 0.80350345  1.03154495  1.6339461  -0.07037684]
Average AUC 0.5927129651428774 [0.6332606085051382, 0.5124299396672414, 0.6324483472562528]
Average AUC 0.5004464141191219 [0.3442625741708125, 0.4957709475887081, 0.6613057205978453]
150 94015.34737693683 -3.448702627892695 3.6461196459534455 [ 0.80165714  1.03291066  1.63710733 -0.0725516 ]
Average AUC 0.5921118152552084 [0.6321941814026114, 0.5125375902313573, 0.6316036741316564]
Average AUC 0.5128877321428136 [0.6548737639289217, 0.5012745153975293, 0.3825149171019898]
155 93293.54729143932 -3.4211969219337073 3.6173083174926774 [ 0.80294666  1.03439137  1.64214654 -0.0709923 ]
Average AUC 0.5914945679582679 [0.6307307361053454, 0.5126374361295747, 0.6311155316398838]
Average AUC 0.6023510282

Average AUC 0.5848515235824961 [0.614617341248379, 0.5193298025742479, 0.6206074269248617]
Average AUC 0.5675574333468769 [0.5493293470000136, 0.4989726099287192, 0.654370343111898]
285 81004.13343079574 -2.8364772392788304 3.0412231732239943 [ 0.79395921  1.09661458  1.68382239 -0.08426776]
Average AUC 0.5848665588997015 [0.6143585918971621, 0.5198401335297598, 0.6204009512721825]
Average AUC 0.5996913591724807 [0.6479324933746633, 0.4930042608093277, 0.6581373233334512]
290 80560.7950438977 -2.8182937535868025 3.021700146450734 [ 0.795287    1.09983308  1.68529159 -0.08334017]
Average AUC 0.5851059764128348 [0.6144897672325321, 0.5205290298584988, 0.6202991321474733]
Average AUC 0.49666391983064634 [0.6541064646086326, 0.4973669344833284, 0.33851836039997785]
295 80258.1736470162 -2.7990344746642646 3.0043269921200446 [ 0.79338923  1.10306383  1.68550964 -0.08516718]
Average AUC 0.5851415147620961 [0.6142286170016331, 0.5210036342830445, 0.6201922930016104]
Average AUC 0.501254925000

Average AUC 0.5916663205257061 [0.6161999574826035, 0.5359618147301497, 0.6228371893643649]
Average AUC 0.5834705636811811 [0.6463220487623028, 0.5012552728591936, 0.6028343694220472]
425 71943.72040039876 -2.3681548681166804 2.5800503647649204 [ 0.78651203  1.2016274   1.7010508  -0.0979198 ]
Average AUC 0.5919241671297485 [0.6163263310586089, 0.5367371006365916, 0.6227090696940449]
Average AUC 0.5089853348108304 [0.6583728278041074, 0.49405849624003495, 0.37452468038834885]
430 71604.95637657618 -2.3538482036641653 2.5646150436089554 [ 0.78763105  1.20586943  1.70170266 -0.09753507]
Average AUC 0.5924691625370438 [0.6169323349166045, 0.5374354163896907, 0.6230397363048367]
Average AUC 0.609235888668918 [0.6656582972000942, 0.5019411415158168, 0.660108227290843]
435 71334.06607793322 -2.33862509899684 2.5508162334175135 [ 0.78619717  1.21013502  1.7016549  -0.09882188]
Average AUC 0.5923347947477692 [0.6160093058096486, 0.5371765840645946, 0.6238184943690641]
Average AUC 0.49951794437

Average AUC 0.6033437718663203 [0.625579430485148, 0.5529350791742986, 0.6315168059395144]
Average AUC 0.3967756888913995 [0.33362089324073796, 0.5039082537302266, 0.3527979197032338]
565 64586.08258011822 -1.984565604834825 2.2032137662578237 [ 0.77950785  1.33007792  1.70696315 -0.11333931]
Average AUC 0.6032099422300179 [0.6247236260092971, 0.5522681839296009, 0.6326380167511558]
Average AUC 0.3874302713920501 [0.33492577134807644, 0.5008067736964459, 0.326558269131628]
570 64348.72322128666 -1.9722800143129506 2.1903160047653225 [ 0.78011165  1.3350043   1.70729666 -0.11347844]
Average AUC 0.6039105401271491 [0.6257458550994902, 0.5532918735127401, 0.632693891769217]
Average AUC 0.38798580426943147 [0.3303702112817773, 0.4973997679053837, 0.33618743362113335]
575 64105.94871631687 -1.9594591400416925 2.1785114772481338 [ 0.77908697  1.33996066  1.70723314 -0.11442571]
Average AUC 0.6042505754941555 [0.6257127884384112, 0.5539293666971138, 0.6331095713469415]
Average AUC 0.502596701

Average AUC 0.6146636180944224 [0.6358133983928075, 0.5663046737030664, 0.6418727821873934]
Average AUC 0.49634858368198637 [0.33035318686221177, 0.506338196088732, 0.6523543680950155]
705 58214.62344226984 -1.653525927368853 1.8803295721574544 [ 0.7711305   1.47558361  1.70944556 -0.12941645]
Average AUC 0.6147095517943081 [0.635727621509612, 0.5663059520535152, 0.642095081819797]
Average AUC 0.3906780494713749 [0.33089502175415253, 0.5045026194073513, 0.336636507252621]
710 58014.272356578316 -1.6426805440053578 1.8695596667936119 [ 0.77104752  1.48104351  1.7094494  -0.12986471]
Average AUC 0.6149822105869706 [0.6359145627321487, 0.5666690708625985, 0.6423629981661644]
Average AUC 0.5008197013721162 [0.668817963992916, 0.4946211050007455, 0.33902003512268714]
715 57783.91135519351 -1.6314872674512695 1.8588882896262588 [ 0.77051815  1.48650506  1.70960931 -0.13049662]
Average AUC 0.6153929985870161 [0.6361599981142181, 0.5671509417002223, 0.6428680559466079]
Average AUC 0.4070421894

Average AUC 0.6244150588491578 [0.6449504916783326, 0.5786572395273494, 0.6496374453417915]
Average AUC 0.5075602751832705 [0.6657546597800708, 0.5029897925735105, 0.35393637319623006]
845 52479.92677780627 -1.362631630115478 1.6009784099206628 [ 0.75938663  1.63316537  1.71067053 -0.14548093]
Average AUC 0.6246647592870294 [0.6452646795240322, 0.5790312579560495, 0.6496983403810065]
Average AUC 0.3901867600339443 [0.3399404450881712, 0.5049411608929184, 0.3256786741207433]
850 52281.8832706408 -1.3532608043633283 1.5916760287843665 [ 0.75931133  1.63892099  1.71067855 -0.14592048]
Average AUC 0.6252176911920961 [0.6454851239312267, 0.5800674619172673, 0.6501004877277944]
Average AUC 0.39308852432704033 [0.32695244992310635, 0.5077651715976905, 0.34454795146032413]
855 52120.27287316302 -1.343500947248338 1.5827937645947328 [ 0.75842691  1.64466093  1.71072278 -0.14663441]
Average AUC 0.6245782431210961 [0.6444828657947546, 0.5787937538989689, 0.6504581096695647]
Average AUC 0.50376857

Average AUC 0.6278876284151899 [0.6452467820573095, 0.5855860335234622, 0.652830069664798]
Average AUC 0.5032285665229287 [0.3451212160499243, 0.5071420766762673, 0.6574224068425945]
985 48063.07313469284 -1.1280586275368616 1.3846012630245448 [ 0.74100609  1.78983699  1.71327802 -0.16095102]
Average AUC 0.6288948951901175 [0.6474779631984432, 0.5882748754886662, 0.6509318468832435]
Average AUC 0.5062279544340554 [0.37701613871322454, 0.5072526203492937, 0.634415104239648]
990 48025.76258427478 -1.1215859185519876 1.3789578676204848 [ 0.74017044  1.79517167  1.71333378 -0.16149037]
Average AUC 0.6276044545313286 [0.6480798200824419, 0.5884682428144594, 0.6462653006970844]
Average AUC 0.5021895410278767 [0.6284571030461925, 0.49200586910875554, 0.3861056509286821]
995 47809.236831733666 -1.11510558721106 1.3735382543310815 [ 0.73910339  1.80048079  1.71348878 -0.16202849]
Average AUC 0.6272950204985179 [0.644839505558473, 0.5850959543303247, 0.6519496016067561]
Average AUC 0.57736642494

In [175]:
calc_metric_auc()

Average AUC 0.5947835819491095 [0.6443484165325448, 0.500163965265469, 0.6398383640493148]


0.5947835819491095